In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

pd.options.display.max_columns = 100
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# load the data (Apr - Dec 2015)
# customer's historical accounts data and payment behavior history
raw_acc_30 = pd.read_csv('./raw_account_30_new.csv') # (186329, 21)

# customer's historical enquiry data such as enquiry amount / purpose
raw_enq_30 = pd.read_csv('./raw_enquiry_30_new.csv') # (413188, 6)

# current customer applications with demographic data
raw_data_30 = pd.read_csv('./raw_data_30_new.csv') # (23896, 83)

/Users/WenLin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/WenLin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,20,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_enq_30.head()

,dt_opened,customer_no,upload_dt,enquiry_dt,enq_purpose,enq_amt
0,21-Apr-15,1,21-Apr-15,10-Nov-14,51.0,100000.0
1,21-Apr-15,1,21-Apr-15,4-Sep-14,12.0,1000000.0
2,21-Apr-15,1,21-Apr-15,4-May-14,51.0,125000000.0
3,21-Apr-15,1,21-Apr-15,19-Dec-13,3.0,400000000.0
4,21-Apr-15,1,21-Apr-15,1-Nov-13,0.0,1000.0


In [4]:
raw_acc_30.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,cur_balance_amt,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
0,01-Oct-15,8226,07-Sep-15,10,1,23-Jul-15,NaN,NaN,20-Aug-15,31937.0,31937,NaN,"""""""000""""""",NaN,01-Aug-15,01-Aug-15,54000.0,10000.0,39,3.0,NaN
1,01-Oct-15,8226,07-Sep-15,10,1,27-Mar-15,09-Aug-15,NaN,22-Aug-15,51311.0,34683,NaN,"""""""000000000000000""""""",NaN,01-Aug-15,01-Apr-15,60000.0,12000.0,NaN,3.0,24012.0
2,01-Oct-15,8226,07-Sep-15,5,1,28-Feb-15,07-Jul-15,NaN,31-Jul-15,150000.0,136642,NaN,"""""""000000000000000""""""",NaN,01-Jul-15,01-Mar-15,NaN,NaN,NaN,3.0,5274.0
3,01-Oct-15,8226,07-Sep-15,10,1,27-Aug-14,26-Jul-15,NaN,31-Jul-15,19000.0,0,NaN,"""""""000000000000000000000000000000000000""""""",NaN,01-Jul-15,01-Aug-14,NaN,NaN,NaN,NaN,NaN
4,29-Jul-15,3439,23-Jul-15,6,1,25-Apr-15,02-Jun-15,NaN,30-Jun-15,18500.0,10276,NaN,"""""""000000000""""""",NaN,01-Jun-15,01-Apr-15,NaN,NaN,NaN,NaN,NaN


In [5]:
raw_data_30.head()

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,Bad_label
0,21-Apr-15,1,17-Apr-15,Insignia,17-Apr-15,726.0,1.0,Card Setup,14.0,1200000.0,NaN,NaN,NaN,Y,IS1,NaN,0.0,157,5107,5107,NaN,2.0,AOUPSXXXXX,9-Sep-59,98220XXXXX,N,@OXFORDGROUP.IN,1.0,3.0,NaN,Pune,411036.0,1999.0,0.0,Self,Y,1.0,200000.0,Private Ltd. Co.,Banking/Financial Services,DIRECTOR,1999.0,0.0,11.0,0.0,Pune,411001.0,@OXFORDGROUP.IN,PAN Card,AOUPSXXXXX,NaN,NaN,N,NaN,0.0,NaN,17-Jun-16,3.0,21.0,NaN,N,Y,N,NaN,Y,1999-0,21.0,16.0,411036.0,0.0,1.0,200000.0,0-0,21.0,R,NaN,NaN,0000-00-00,0.0,98220XXXXX,1.0,N,0
1,12-May-15,2,6-May-15,Insignia,6-May-15,844.0,1.0,Card Setup,14.0,300000.0,NaN,NaN,NaN,Y,IS1,NaN,0.0,167,5639,5639,NaN,2.0,ALJPSXXXXX,2-Oct-51,98308XXXXX,N,@GMAIL.COM,1.0,0.0,NaN,Kolkata / Howrah,700019.0,1964.0,0.0,Self,Y,2.0,70000.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,Kolkata,700019.0,NaN,NaN,XXXXX,The Ratnakar Bank Ltd.,NaN,N,NaN,0.0,NaN,17-Jun-16,3.0,35.0,NaN,Y,Y,N,NaN,Y,1964-0,35.0,58.0,700019.0,0.0,2.0,70000.0,Nov-00,35.0,R,NaN,NaN,0000-00-00,0.0,98308XXXXX,1.0,N,0
2,21-May-15,3,20-May-15,Insignia,20-May-15,723.0,3.0,Card Setup,14.0,1500000.0,NaN,NaN,NaN,Y,IS2,NaN,0.0,116,5378,5378,NaN,1.0,ABAPKXXXXX,2-Apr-70,98107XXXXX,N,NaN,1.0,0.0,NaN,New Delhi,110017.0,1970.0,0.0,Self,Y,2.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,New Delhi,110017.0,@TRIFECTACAPITAL.IN,NaN,XXXXX,The Ratnakar Bank Ltd.,NaN,N,NaN,0.0,NaN,17-Jun-16,1.0,10.0,NaN,Y,Y,N,NaN,Y,1970-0,10.0,157.0,110017.0,0.0,2.0,1.0,Nov-00,10.0,R,NaN,NaN,0000-00-00,0.0,98107XXXXX,1.0,N,0
3,22-May-15,4,22-May-15,Insignia,22-May-15,778.0,1.0,Card Setup,14.0,2500000.0,NaN,NaN,NaN,Y,IS2,NaN,0.0,182,D068,5627,NaN,1.0,AAFPKXXXXX,24-Dec-74,98100XXXXX,N,@KAJARIACERAMICS.COM,1.0,3.0,NaN,New Delhi,110062.0,1974.0,0.0,Self,Y,2.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,New Delhi,110044.0,@KAJARIACERAMICS.COM,PAN Card,AAFPKXXXXX,NaN,NaN,N,NaN,0.0,NaN,17-Jun-16,1.0,10.0,NaN,N,Y,N,NaN,Y,1974-0,10.0,157.0,110062.0,0.0,2.0,1.0,Nov-00,10.0,O,NaN,NaN,0000-00-00,0.0,98100XXXXX,1.0,N,0
4,22-May-15,5,12-May-15,Insignia,13-May-15,810.0,1.0,Card Setup,14.0,500000.0,NaN,NaN,NaN,Y,IS1,NaN,0.0,155,5109,D510,NaN,1.0,AAAPDXXXXX,12-Sep-51,98202XXXXX,N,@GMAIL.COM,1.0,1.0,NaN,Mumbai / Navi Mumbai / Thane,400018.0,2000.0,0.0,Self,Y,2.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,Mumbai,400034.0,@JDVAUTO.COM,NaN,XXXXX,The Ratnakar Bank Ltd.,NaN,Y,NaN,0.0,NaN,17-Jun-16,1.0,21.0,NaN,Y,Y,Y,NaN,Y,2000-0,21.0,15.0,400018.0,0.0,2.0,1.0,Nov-00,21.0,R,NaN,NaN,1/31/44,1.0,98202XXXXX,2.0,N,0


In [6]:
# Find a way to merge all 3 dataframes together 
df_acc = raw_acc_30.copy()
df_enq = raw_enq_30.copy()
df_data = raw_data_30.copy()

In [7]:
def pay_clean(data):
    output = []
    data = data[3:-3] # to get rid of """ """

    for i in range(0,len(data)+3,3): # count +3
        output.append(data[i:i+3])
    return output

def count_dpd(data):
    for i in sorted(data,reverse=True):
        if i in ['XXX','STD','SMA','SUB','DBT','LSS']:
            pass
        else:
            break
    return i

## Feature Extraction/Engineering - Accounts Data

- **Payment history behavior in 0-29 DPD Bucket**

In [8]:
a1 = df_acc[['customer_no','paymenthistory1']]
a1['paymenthistory1'] = a1['paymenthistory1'].apply(pay_clean).apply(list)
a1['dpd'] = a1['paymenthistory1'].apply(count_dpd)

# mean counts of payment history behavior within 000-029 dpd bucket
a1['dpd_29']=a1['dpd'].apply(lambda x: 1 if x <= '029' and x >= '000' else 0)
a1 = a1.groupby('customer_no').mean()
a1.reset_index(drop=True, inplace=True)

In [9]:
a1.head()

,dpd_29
0,0.625000
1,1.000000
2,0.727273
3,0.840000
4,0.833333


As can be seen from this distribution, clearly that majority of the customers do make their payments on time before it reaches 30 days pass due. 

- **Total duration between last payment date and account opened date**

In [10]:
a2 = df_acc[['customer_no','last_paymt_dt','opened_dt']]
a2['opened_dt'] = pd.to_datetime(a2['opened_dt'])
a2['last_paymt_dt'] = pd.to_datetime(a2['last_paymt_dt'])

a2['total_lastpy_opendt'] = a2['last_paymt_dt'] - a2['opened_dt']
a2['total_lastpy_opendt'] = a2['total_lastpy_opendt'].astype('timedelta64[D]')

In [11]:
a2 = a2.groupby('customer_no').sum()
a2 = a2['total_lastpy_opendt']

a2.reset_index(drop=True, inplace=True)
a2.head()

0    14207.0
1      817.0
2    17867.0
3    35033.0
4    27870.0
Name: total_lastpy_opendt, dtype: float64

In [12]:
# replace missing values
a2 = a2.fillna(a2.median())

Distribution is positively skewed.

- **Smallest number of months passed before first 30 DPD appeared**

In [13]:
def minmonth(data):
    output = 0
    for i in data:
        if (i < '030') or i in ['XXX','STD','SMA','SUB','DBT','LSS']:
            output+=1 # each [000] represents a month so in paymenthistory1
        else:
            return output
    return 0

In [14]:
a3 = df_acc[['customer_no','paymenthistory1']]
a3['paymenthistory1'] = a3['paymenthistory1'].apply(pay_clean).apply(list)

a3['min_months_last_30_plus'] = a3['paymenthistory1'].apply(minmonth)
a3 = a3.groupby('customer_no').sum()
a3.reset_index(drop=True, inplace=True)
a3.head()

,min_months_last_30_plus
0,18
1,0
2,2
3,0
4,0


- **Ratio of total current bal amt / total credit limit**

In [15]:
a4 = df_acc[['customer_no','cur_balance_amt','creditlimit']]
a4 = a4.groupby('customer_no')['cur_balance_amt','creditlimit'].apply(sum).reset_index()
a4['creditlimit'] = a4['creditlimit'].replace(0, np.inf) 
a4['total cur_bal_amt/total credit limit'] = a4['cur_balance_amt'] / a4['creditlimit']
a4 = a4[['customer_no','total cur_bal_amt/total credit limit']]

In [16]:
# for merging columns
a4m = a4['total cur_bal_amt/total credit limit']

- **Utilisation trend**

In [17]:
# determine ultisation trend
a5 = df_acc[['customer_no','creditlimit','cashlimit','cur_balance_amt']]
a5 = a5.fillna(0)

a5 = a5.groupby('customer_no')['cur_balance_amt','creditlimit','cashlimit'].agg(lambda x: list(x))
a5 = a5.applymap(np.mean).reset_index()
a5 = a5.replace(0, np.inf)

a5['mean cur_bal_amt/(mean_credit limit+ mean_cashlimit)'] = a5['cur_balance_amt'] / (a5['creditlimit']/a5['cashlimit'])
a5 = a5[['customer_no','mean cur_bal_amt/(mean_credit limit+ mean_cashlimit)']]
a5 = pd.merge(a5,a4, how='inner', on='customer_no')

a5 = a5.fillna(0)
a5['mean cur_bal_amt/(mean_credit limit+ mean_cashlimit)'] = a5['mean cur_bal_amt/(mean_credit limit+ mean_cashlimit)'].replace(0, np.inf)

a5['utilisation_trend'] = a5['total cur_bal_amt/total credit limit'] / a5['mean cur_bal_amt/(mean_credit limit+ mean_cashlimit)']

a5 = a5[['customer_no','utilisation_trend']]

In [18]:
# for merging
a5m = a5['utilisation_trend']

Majority of the customers maintain a low utilisation rate at around 3% , which means that they maintain a good credit score

- ** Mean difference between last payment date and account open date **

In [19]:
a6 = df_acc[['customer_no','last_paymt_dt','opened_dt']]

a6['opened_dt'] = pd.to_datetime(a6['opened_dt'])
a6['last_paymt_dt'] = pd.to_datetime(a6['last_paymt_dt'])

a6['mean_lastpy_opendt'] = a6['last_paymt_dt'] - a6['opened_dt']
a6['mean_lastpy_opendt'] = a6['mean_lastpy_opendt'].astype('timedelta64[D]')

In [20]:
a6 = a6.groupby('customer_no').mean()
a6 = a6['mean_lastpy_opendt']
a6 = a6.fillna(a6.median())

a6.reset_index(drop=True, inplace=True)
a6.head()

0    1291.545455
1     817.000000
2    1786.700000
3    1592.409091
4    6967.500000
Name: mean_lastpy_opendt, dtype: float64

- **Payment history mean length**

In [21]:
a7 = df_acc[['customer_no','paymenthistory1']]

a7['paymenthistory1'] = a7['paymenthistory1'].apply(pay_clean)
a7['mean_paymt_his'] = a7['paymenthistory1'].apply(len) # each [000] in paymenthistory is 1 month
a7 = a7.groupby('customer_no').mean()

a7.reset_index(drop=True, inplace=True)
a7.head()

,mean_paymt_his
0,14.687500
1,19.000000
2,17.000000
3,14.080000
4,16.166667


In [22]:
# replace 19 with median value
median = a7.loc[a7['mean_paymt_his'] < 19, 'mean_paymt_his'].median()
a7['mean_paymt_his'] = a7["mean_paymt_his"].mask(a7["mean_paymt_his"] == 19, median)

Distribution is more normal now.

## Feature Extraction/Engineering - Enquiry Data

In [23]:
df_enq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173499 entries, 0 to 173498
Data columns (total 6 columns):
dt_opened      173499 non-null object
customer_no    173499 non-null int64
upload_dt      173457 non-null object
enquiry_dt     173457 non-null object
enq_purpose    173457 non-null float64
enq_amt        173457 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 7.9+ MB


- **Enquiry count in pass 365 days and 90 days**

In [24]:
def days_pass_enq_dt(data):
    data['enquiry_dt'] = pd.to_datetime(data['enquiry_dt'])
    data['dt_opened'] = pd.to_datetime(data['dt_opened'])
    
    # No. of days pass enquiry date
    data['days_pass_enq_dt'] = data['dt_opened'] - data['enquiry_dt']
    data['days_pass_enq_dt'] = data['days_pass_enq_dt'].astype('timedelta64[D]')
    return data

df_enq_days = days_pass_enq_dt(df_enq)

In [25]:
# Retreive enquiry dataframe made in pass 365 days and 90 days
b1 = df_enq_days[df_enq_days.days_pass_enq_dt <= 365].groupby('customer_no').size()
b2 = df_enq_days[df_enq_days.days_pass_enq_dt <= 90].groupby('customer_no').size() 

- **Most frequent enquiry purpose for each customer**

In [26]:
# Retrieve most frequent enquiry purpose for each customer
def max_freq_enq(data):
    data = pd.DataFrame(data.groupby('customer_no').enq_purpose.apply(lambda x: stats.mode(x)[0]))
    data['enq_purpose'] = data['enq_purpose'].str[0] 
    data = data.rename(index=str, columns={"enq_purpose": "max_freq_enq"})
    data.reset_index(drop=True, inplace=True)
    return data
b3 = max_freq_enq(df_enq) 

- **Ratio of secured loan type over total enquiry purpose made**

In [27]:
def sec_loan(data):
    # Retrieve enquiry dataframe where enquiry purpose is a secured loan
    sec = data.enq_purpose
    m1 = (sec == 1 )|(sec == 2 )|(sec == 3 )|(sec == 4 )|(sec == 7 )|(sec == 11)|(sec == 13)|(sec == 14)|(sec == 15)|(sec == 17)
    m2 = (sec == 32)|(sec == 33)|(sec == 34)|(sec == 51)|(sec == 52)|(sec == 53)|(sec == 54)|(sec == 55)|(sec == 56)|(sec == 57)
    m3 = (sec == 58)|(sec == 59)
    mask = m1|m2|m3
    return data[mask]

df_enq_sec = sec_loan(df_enq)

In [28]:
# Find ratio of secured loan enquiry purpose/ total enquiry purpose made
total_enq = df_enq.groupby('customer_no').size()
sec_enq = df_enq_sec.groupby('customer_no').size()

b4 = (sec_enq/total_enq).T

- **Average difference between enquiry dt_opened and enquiry_dt**

In [29]:
# Determine average difference between enquiry dt_opened and enquiry_dt
b5 = df_enq_days.groupby('customer_no').mean()
b5 = b5.drop(['enq_purpose','enq_amt'], axis=1)
b5.reset_index(drop=True, inplace=True)

## Feature Extraction/Engineering - Demographic Data 

In [30]:
df_data = raw_data_30.copy()

In [31]:
df_data.shape

(10240, 83)

**Exploratory Data Analysis**

In [32]:
df_data.isnull().sum().sort_values(ascending=False)

feature_61     10234
feature_74     10233
feature_18     10227
feature_10     10220
feature_49     10185
feature_17      9812
feature_8       9708
feature_9       9708
feature_57      9202
feature_73      8966
feature_48      7966
feature_45      5905
feature_13      5518
feature_53      4984
feature_51      4916
feature_14      3263
feature_38      2448
feature_37      2448
feature_36      2448
feature_27      1533
feature_3       1211
feature_2       1211
feature_24       276
feature_46        50
feature_15        14
feature_16        12
feature_30         9
feature_29         9
feature_31         9
feature_32         9
               ...  
feature_70         9
feature_69         9
feature_68         9
feature_67         9
feature_66         9
feature_65         9
feature_64         9
feature_63         9
feature_62         9
feature_60         9
feature_59         9
feature_58         9
feature_56         9
feature_55         9
feature_54         9
feature_52         9
feature_50   

In [33]:
# Remove features with >50% null values
drop_feats_null = ['feature_61','feature_74','feature_18','feature_10','feature_49','feature_17','feature_8','feature_9',
                   'feature_57','feature_73','feature_48','feature_45','feature_13','feature_53','feature_51']
df_data = df_data.drop(drop_feats_null, axis=1)

In [34]:
def nearZeroVariance(X, freqCut = 95 / 5, uniqueCut = 10):
    '''
    Determine predictors with near zero or zero variance.
    Inputs:
    X: pandas data frame
    freqCut: the cutoff for the ratio of the most common value to the second most common value
    uniqueCut: the cutoff for the percentage of distinct values out of the number of total samples
    Returns a tuple containing a list of column names: (zeroVar, nzVar)
    '''
    
    colNames = X.columns.values.tolist()
    freqRatio = dict()
    uniquePct = dict()

    for names in colNames:
        counts = (
            (X[names])
            .value_counts()
            .sort_values(ascending = False)
            .values
            )

        if len(counts) == 1:
            freqRatio[names] = -1
            uniquePct[names] = (len(counts) / len(X[names])) * 100
            continue

        freqRatio[names] = counts[0] / counts[1]
        uniquePct[names] = (len(counts) / len(X[names])) * 100

    zeroVar = list()
    nzVar = list()
    for k in uniquePct.keys():
        if freqRatio[k] == -1:
            zeroVar.append(k)

        if uniquePct[k] < uniqueCut and freqRatio[k] > freqCut:
            nzVar.append(k)
    print "features with zero variance:",zeroVar
    print "features with nearZero variance",nzVar
    print "removing features with zero/nearZero variance from dataset..."

    return(X.drop(zeroVar+nzVar, axis=1))

In [35]:
# Remove features with near zero, zero variance
df_data = nearZeroVariance(df_data)

features with zero variance: ['feature_6', 'feature_5', 'feature_54']
features with nearZero variance ['feature_31', 'feature_75', 'feature_76', 'feature_46', 'feature_60', 'feature_62', 'feature_79', 'feature_78', 'feature_42', 'feature_58', 'Bad_label', 'feature_55']
removing features with zero/nearZero variance from dataset...


In [36]:
# Added back the target feature
df_data.Bad_label = raw_data_30.Bad_label

# Customers who defaulted
df_data_def = df_data[df_data.Bad_label == 1]

In [37]:
# Remove unnecessary features
unnecessary_feats = ['dt_opened','entry_time','feature_2','feature_4','feature_7','feature_11','feature_12','feature_14',
                     'feature_15','feature_19','feature_16','feature_20','feature_22','feature_23','feature_25','feature_67',
                     'feature_24','feature_26','feature_28','feature_29','feature_30','feature_33','feature_50','feature_59',
                     'feature_34','feature_35','feature_3','feature_39','feature_40','feature_41','feature_44','feature_47',
                     'feature_52','feature_56','feature_63','feature_64','feature_65','feature_66','feature_68','feature_69',
                     'feature_70','feature_71','feature_77','feature_72']
df_data = df_data.drop(unnecessary_feats, axis=1)

I've made an assumption that feature_21 is the date of birth of the customer and proceeded to determine their age as of Dec 2015. 

In [38]:
from datetime import timedelta, date
col = 'feature_21'
df_data[col] = pd.to_datetime(df_data[col])
future = df_data[col] > date(year=2050,month=1,day=1)
df_data.loc[future, col] -= timedelta(days=365.25*100)

# determine age of customer from Dec 2015
df_data['age'] = pd.to_datetime('2015-12-31') - df_data['feature_21']
df_data['age'] = df_data['age'].astype('timedelta64[Y]')
df_data['age'] = df_data['age'].fillna(df_data['age'].median())

In [39]:
df_data.age.describe()

count    10240.000000
mean        33.934082
std          7.575247
min        -34.000000
25%         29.000000
50%         32.000000
75%         37.000000
max         64.000000
Name: age, dtype: float64

In [40]:
# replace negative age values 
df_data.loc[df_data['customer_no'] == 24, 'age'] = 66
df_data.loc[df_data['customer_no'] == 5823, 'age'] = 67
df_data.loc[df_data['customer_no'] == 5560, 'age'] = 67
df_data.loc[df_data['customer_no'] == 9476, 'age'] = 68
df_data.loc[df_data['customer_no'] == 5030, 'age'] = 73

In [41]:
df_data.age.describe()

count    10240.000000
mean        33.983105
std          7.471286
min         21.000000
25%         29.000000
50%         32.000000
75%         37.000000
max         73.000000
Name: age, dtype: float64

Label encoding categorical features in the dataset.

In [42]:
df_data = df_data.fillna(0)

In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
def encode_with_LabelEncoder(df, column_name):
    label_encoder = LabelEncoder()
    label_encoder.fit(df[column_name])
    df[column_name+'_le'] = label_encoder.transform(df[column_name])
    df.drop([column_name], axis=1, inplace=True)
    return label_encoder

In [45]:
# label encoding categorical features    
encode_with_LabelEncoder(df_data,'feature_43')
encode_with_LabelEncoder(df_data,'feature_38')

df_data = pd.get_dummies(data=df_data, columns=['feature_1','feature_27','feature_32','feature_36','feature_37'])

In [46]:
df_data = df_data.drop(['feature_21','feature_1_0','feature_27_0',
                        'feature_32_0','feature_36_0','feature_37_0'],axis=1)

In [47]:
df_data['Bad_label'] = raw_data_30.Bad_label

In [48]:
df_data.shape

(10240, 46)

In [49]:
# Prepare data for merging
df_data = df_data.drop(['customer_no','Bad_label'],axis=1)

## Final Data Model - for training 

In [50]:
# Import customer_no
a0 = raw_data_30.customer_no
d0 = raw_data_30.Bad_label
# Generate feature engineered data model for modeling
data_model = pd.concat([a0,a1,a2,a3,a4m,a5m,a6,a7,b1,b2,b3,b4,b5,df_data,d0],axis=1)
data_model[[0,1,2]] = data_model[[0,1,2]].shift(-1) 
data_model = data_model.rename(index=str, columns={0:"no_enq_365", 1:"no_enq_90", 2:"ratio_sec/tot_enq"})
# Remember to drop last row
data_model = data_model.drop(data_model.index[10240])
data_model

,customer_no,dpd_29,total_lastpy_opendt,min_months_last_30_plus,total cur_bal_amt/total credit limit,utilisation_trend,mean_lastpy_opendt,mean_paymt_his,no_enq_365,no_enq_90,max_freq_enq,ratio_sec/tot_enq,days_pass_enq_dt,age,feature_43_le,feature_38_le,feature_1_Golf Card,feature_1_Insignia,feature_1_Platinum Cricke,feature_1_Platinum Deligh,feature_1_Platinum Maxima,feature_1_RBL Bank Fun+,feature_1_Titanium Deligh,feature_27_CA,feature_27_Diploma,feature_27_Doctor,feature_27_Engineer,feature_27_Graduate,feature_27_Lawyer,feature_27_MBA/MMS,feature_27_Others,feature_27_Post-Graduate,feature_27_Professional,feature_32_Compa,feature_32_PG/Ho,feature_32_Paren,feature_32_Rente,feature_32_Self,feature_36_Government Organisation,feature_36_MNC,feature_36_PSU,feature_36_Partnership,feature_36_Partnership Co.,feature_36_Private Ltd. Co.,feature_36_Proprietorship,feature_36_Public Ltd Co.,feature_37_Agriculture,feature_37_Banking/Financial Services,feature_37_Health Care,feature_37_Industrial,feature_37_Information Technology,feature_37_KPO/LPO/ITES,feature_37_Manufacturing,feature_37_Others,feature_37_Real Estate,feature_37_Tourism,feature_37_Transportation/Logistics,Bad_label
0,1.0,0.625000,14207.0,18.0,29.673733,0.000057,1291.545455,14.687500,3.0,NaN,2.0,0.696970,1343.575758,56.0,39.0,820.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1.000000,817.0,0.0,-0.000000,-0.000000,817.000000,12.600000,NaN,NaN,10.0,NaN,1370.500000,64.0,29.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.727273,17867.0,2.0,19.404914,0.000021,1786.700000,17.000000,1.0,1.0,2.0,0.470588,1914.705882,45.0,36.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.840000,35033.0,0.0,2.466711,0.000014,1592.409091,14.080000,2.0,NaN,5.0,0.380000,2016.400000,41.0,36.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.833333,27870.0,0.0,0.120417,0.000026,6967.500000,16.166667,2.0,NaN,51.0,0.600000,633.800000,64.0,33.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6.0,0.500000,6637.0,15.0,16.302214,0.000046,1659.250000,12.600000,2.0,NaN,10.0,0.384615,1128.307692,27.0,39.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7.0,1.000000,751.0,0.0,0.126017,0.000000,375.500000,14.000000,3.0,NaN,10.0,0.111111,453.666667,42.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8.0,0.777778,18975.0,1.0,0.445266,0.000034,1459.615385,11.555556,1.0,1.0,10.0,NaN,1905.454545,57.0,29.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9.0,0.777778,4695.0,8.0,0.061667,0.000450,782.500000,8.444444,2.0,NaN,10.0,0.235294,1894.588235,44.0,33.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10.0,0.888889,30649.0,10.0,2.548681,0.000242,1277.041667,14.555556,8.0,NaN,5.0,0.070000,1779.680000,34.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [51]:
data_model = data_model.to_csv('data_model_test.csv')